# 7 - Question answering

There are many flavours of question answering (QA), but the most common is extractive QA, which involves questions whose answer can be identified as a psan of text in a document, where the document might be a web page, legal contract, or news article. The two-stage process of first retrieving relevant documents and then extracting answers from them is also the basis for many modern QA systems, including:

* semantic search engines
* intelligent assistants
* automated information extractors

In this chapter, we'll apply this process to tackle a common problem facing ecommerce websites: helping consumers answer specific queries to evaluate a product. We'll see that customer reviews can be used as a rich and challenging source of information for QA, and along the way we'll learn how transformers act as a powerful *reading comprehension* models that can extract meaning from text. 

----

**Note:** This chapter focuses on <span style="color:blue"><b>extractive QA</b></span>, but other forms of QA may be more suitable for your use case. For example, <span style="color:blue">community QA</span> involves gathering question-answer pairs that are generated by users on forums like Stack Overflow, and then using semantic similarity search to find the closest matching answer to a new question. There is also <span style="color:blue">long-form QA</span>, which aims to generate complex paragraph-length answers to open-ended questions like "Why is the sky blue?" Remarkably, it is also possible to do QA over tables, and transformer models like TAPAS can even perform aggregations to produce the final answer!

----



## 7.1 - Building a review-based QA system

If you have ever purchased a product online, you probably relied on customer reviews to help inform your decision. These reviews can often help answer specific questions like "Does this guitar come with a strap?" or "Can I use this camera at night?" that may be hard to answer from the product description alone. However, popular products can have hundreds to thousands of reviews, so it can be a major drag to find one that is relevant. One alternative is to post your question on the community QA platform provided by websites like Amazon, but it usually takes days to get an answer (if you get one at all). Wouldn't it be nice if we could get an immediate answer like Google sometimes provides with its search engine?

<img src="images/google_search_qa.png" title="" alt="" width="700" data-align="center">


### 7.1.1 - The dataset

To build our QA system we'll use the [SubjQA dataset](https://arxiv.org/abs/2004.14283), which consists of more than 10000 customer reviews in English about products and services in six domains: Trip-Advisor, Restaurants, Movies, Books, Electronics, and Grocery. As illustrated in the following figure, each review is associated with a question that can be answered using one or more sentences from the review.

<img src="images/qa_product_example.png" title="" alt="" width="600" data-align="center">

the interesting aspect of this dataset is that most of the questions ansd answers are *subjective*; that is, they depend on the personal experience of the users. This example shows why this feature makes the task potentially more difficult than finding answers to factual questions like "What is the currency of the United Kingdom?" First, the query is about "poor quality", which is subjective and depends on the user's definition of quality. Second, important parts of the query do not appear in the review at all, which means it cannot be answered with shortcuts like keyword search or paraphrasing the input question. These features make SubjQA a realistic dataset to benchmark our review-based QA models on, since user-generated content like that resembles what we might encounter in the wild.

----

**Note:** QA systems are usually categorized by the domain of data that they have access to when responding to a query. *Closed-domain QA* deals with questions about a narrow topic (e.g., a single product category), while *open-domain QA* deals with questions about almost anything (e.g., Amazon's whole product catalog). In general, closed-domain QA involves searching through fewer documents than the open-domain case.

----

Let's begin by downloading the dataset from the Hugging Face Hub. We can use the `get_dataset_config_names()` function to find out which subsets are available:

In [ ]:
from datasets import get_dataset_config_names

domains = get_dataset_config_names("subjqa")
domains

For our use case, we'll focus on building a QA system for the Electronics domain. To download the `electronics` subset, we just need to pass this value to the `name` argument of the `load_dataset()` function:

In [ ]:
from datasets import load_dataset

subjqa = laod_dataset("subjqa", name="electronics")

Like other question answering datasets on the Hub, SubjQA stores the answers to each question as a nested dictionary. For example, if we inspect one of the rows in the `answers` column:

In [ ]:
print(subjqa["train"]["answers"][1])

we can see that the answers are stored in a `text` field, while